# Cython...
czyli jak bezbolesnie przyspieszyc kod 100 razy

!= CPython

prawie CPython (99% kompatybilnosci) + rozszerzenia

http://cython.org

jak przygotujemy pakiet w Cythonie albo modul,
to moze on zostac bez problemu zaladowany w normalnym python

* daje dostep do wywolan funkcji w C i C++ i Fortranie z kodu
* kod Cythona kompilowany jest do C/C++ i to jest kompilowane za pomoca gcc/clang do ladowanej dynamicznie biblioteki (.dll/.so)
* dobrze zintegrowany tez numpy

Zadanie: napisz funkcje pi1() ktora przybliza wartosc pi, dla danego n, wg nastepujacego wzoru:
$$
\pi \simeq 4 \sum_{i=0}^n \frac{(-1)^i}{2i+1}
$$
(wzor Gregory'ego-Leibniza)

In [ ]:
def pi1(n):
    """
    wersja pure Python
    """
    s = 1
    a = 1
    for i in range(1, n+1):
            a = -a
            s += a/(2*i+1)
    return 4*s

In [ ]:
pi1(10000)

In [ ]:
import numpy as np
def pi2(n):
    """
    wersja numpy-style (vectorized)
    """
    x = np.arange(0, n+1)
    x = 1/(2*x+1)
    x[1::2] *= -1
    return 4*x.sum()

In [ ]:
pi2(10000)

In [ ]:
%timeit pi1(1000000)
%timeit pi2(1000000)

Wlaczamy obsluge Cython-a w Jupyterze

In [ ]:
%load_ext Cython

In [ ]:
%%cython
def pi3(n):
    s = 1
    a = 1
    for i in range(1, n+1):
            a = -a
            s += a/(2*i+1)
    return 4*s

In [ ]:
%timeit pi1(1000000)
%timeit pi3(1000000)

Ciekawostka - w Jupyterze sprobujmy
%%cython -a

In [ ]:
%%cython -a
def pi3(n):
    s = 1
    a = 1
    for i in range(1, n+1):
            a = -a
            s += a/(2*i+1)
    return 4*s

Optymalizacja kodu polega w wiekszosci na:
* deklaracji typowanych zmiennych

Zlota regula: kazda zmienna ktora pojawia sie w funkcji powinna miec okreslony typ.

Nazwy typow prawie jak z C: int, double

w powyzszym kodzie mamy 4 zmienne: n,s,a,i (nie zapominiamy o i)

In [ ]:
%%cython -a
def pi4(unsigned int n):
    cdef double s = 1
    cdef double a = 1
    cdef unsigned int i
    for i in range(1, n+1):
            a = -a
            s += a/(2*i+1)
    return 4*s

In [ ]:
%timeit pi1(1000000)
%timeit pi4(1000000)

Zadanie: napisz funkcje w pythonie i cythonie do liczenia n-tej liczby fibonacciego

uwaga: int tutaj to nie bigint z pythona (mozna sprobowac double)

Zadanie: Napisz funkcje sum1 przy uzyciu petli for,
ktora liczy sume elementow w danym obiekcie iterowalnym

In [ ]:
def sum1(x):
    s = 0
    n = len(x)
    for i in range(0, n):
            s += x[i]
    return s

In [ ]:
np.random.seed(123)
x = np.random.random(100000)
sum1(x)

In [ ]:
def sum2(x):
    s = 0
    for e in x:
        s = s + e
    return s

In [ ]:
%timeit sum1(x)
%timeit sum2(x)

In [ ]:
sum2([1,2,3,4])

In [ ]:
sum2((1,2,3))

In [ ]:
sum2(range(1, 10))

zalozmy ze robimy wersje python tylko dla wektorow o wartosciach rzeczywistych

np.ndarray[np.double_t]

In [ ]:
%%cython -a
cimport numpy as np
import numpy as np
cimport cython

@cython.boundscheck(False)
def sum3(np.ndarray[np.double_t] x):
    cdef np.double_t s = 0
    cdef unsigned int i, n = len(x)
    for i in range(0, n):
            s += x[i]
    return s

@cython.boundscheck(False)
def sum4(np.ndarray[np.double_t] x):
    cdef np.double_t s = 0
    cdef np.double_t e
    for e in x:
        s = s + e
    return s

In [ ]:
# Moral z tej bajki: niskopoziomowo

In [ ]:
%timeit sum1(x)
%timeit sum2(x)
%timeit sum3(x)
%timeit sum4(x)

In [ ]:
# auto sprawdzanie typow!
sum3(np.arange(0, 10))

In [ ]:
sum3(np.arange(0, 10).astype(float))

In [ ]:
# zadanie: splot 2 macierzy numpy, C=A*B
def convolve1(A, B):
    C = A.copy()
    n, m = A.shape
    k = B.shape[0]//2
    for i in range(k, n-k):
        for j in range(k, m-k):
            C[i,j] = 0
            for ai in range(0, 2*k+1):
                for aj in range(0, 2*k+1):
                    C[i,j] += A[i+ai-k,j+aj-k]*B[ai,aj]
    return C

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import skimage.data
A = skimage.data.lena().mean(axis=2)
B = np.ones((7,7))/(7*7)
C = convolve1(A, B)

fig, (ax0, ax1) = plt.subplots(1,2)
ax0.imshow(A, cmap=plt.cm.gray)
ax0.axis('off')

ax1.imshow(C, cmap=plt.cm.gray)
ax1.axis('off')

plt.show()

In [ ]:
%%cython
import numpy as np
cimport numpy as np
cimport cython

@cython.boundscheck(False)
def convolve2(np.ndarray[np.double_t,ndim=2] A, 
              np.ndarray[np.double_t,ndim=2] B):
    cdef np.ndarray[np.double_t,ndim=2] C = A.copy()
    cdef unsigned int n, m
    # 99% procent kompatybilnosci:
    # n,m = A.shape nie dziala
    n = A.shape[0]
    m = A.shape[1]
    cdef unsigned int k = B.shape[0]//2
    cdef unsigned int i, j, ai, aj
    for i in range(k, n-k):
        for j in range(k, m-k):
            C[i,j] = 0
            for ai in range(0, 2*k+1):
                for aj in range(0, 2*k+1):
                    C[i,j] += A[i+ai-k,j+aj-k]*B[ai,aj]
    return C

In [ ]:
%timeit convolve1(A,B)
%timeit convolve2(A,B)

In [ ]:
%%cython
cimport numpy as np

def f2(np.double_t x):
    return x**2

def test(np.ndarray[np.double_t] t, f):
    # f nietypowane...
    cdef unsigned int n = len(t), i
    cdef double s = 0
    for i in range(0, n):
        s += f(t[i])
    return s

In [ ]:
x = np.random.random(10000)
test(x, f2)

In [ ]:
%timeit test(x, f2)
%timeit sum([xi**2 for xi in x])

In [ ]:
def f4(x):
    return x**2

In [ ]:
test(x, f4)

In [ ]:
%timeit test(x, f2)
%timeit test(x, f4)

## Cython w modulach i pakietach pythona

Stworzmy plik "modul_cython_test.pyx"
ktory zawiera cythonowe definicje funkcji convolve, sum, pi (w katalogu biezacym)

In [ ]:
import os
os.getcwd()

```python
# cython: boundscheck=False, wraparound=False, nonecheck=False
import numpy as np
cimport numpy as np
cimport cython


def convolve2(np.ndarray[np.double_t,ndim=2] A,
              np.ndarray[np.double_t,ndim=2] B):
    cdef np.ndarray[np.double_t,ndim=2] C = A.copy()
    cdef unsigned int n, m
    # 99% procent kompatybilnosci:
    # n,m = A.shape nie dziala
    n = A.shape[0]
    m = A.shape[1]
    cdef unsigned int k = B.shape[0]//2
    cdef unsigned int i, j, ai, aj
    for i in range(k, n-k):
        for j in range(k, m-k):
            C[i,j] = 0
            for ai in range(0, 2*k+1):
                for aj in range(0, 2*k+1):
                    C[i,j] += A[i+ai-k,j+aj-k]*B[ai,aj]
    return C

def sum3(np.ndarray[np.double_t] x):
    cdef np.double_t s = 0
    cdef unsigned int i, n = len(x)
    for i in range(0, n):
            s += x[i]
    return s

def pi4(unsigned int n):
    cdef double s = 1
    cdef double a = 1
    cdef unsigned int i
    for i in range(1, n+1):
            a = -a
            s += a/(2*i+1)
    return 4*s


```

In [2]:
# robimy raz
import pyximport
import numpy as np
pyximport.install(setup_args={'include_dirs': np.get_include()})
# uzywamy tutaj numpy, trzeba dodac specjalne flagi dla kompilatora
# (katalog z plikami .h -- specyfikacja jest podobna jak w pliku setup.py ponizej)

(None, <pyximport.pyximport.PyxImporter at 0x7fd7e16ffb00>)

In [3]:
import modul_cython_test as mct

In [ ]:
mct.pi4(10000)

TODO: czy cython + jupyter dziala pod Windows??

## Pakiety z Cythonem:

3 pliki:

`pakiet_cython/modul_cython_test.pyx`:

jw.

`pakiet_cython/__init__.py`:
```python
from . import modul_cython_test
```

`pakiet_cython/setup.py`:
```python
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext
import numpy as np

setup(
    name = 'Moj pierwszy pakiet Cython',
    include_dirs = [np.get_include()],
    ext_modules = [
        Extension("modul_cython_test",
                   ["modul_cython_test.pyx"]
                 )
    ],
    cmdclass = {
        "build_ext": build_ext
    }
)
```

W terminalu wchodzimy do katalogu,
w ktorym jest plik setup.py

Pakiet trzeba "zbudowac" przed importem

cd costam/pakiet_cython
/opt/anaconda3/bin/python3 setup.py build_ext --inplace

In [ ]:
import pakiet_cython
pakiet_cython.modul_cython_test.pi4(10000)

## OPENMP

```python
# cython: boundscheck=False, wraparound=False, nonecheck=False
import numpy as np
cimport numpy as np
cimport cython
cimport cython.parallel
cimport openmp

def convolve3(np.ndarray[np.double_t,ndim=2] A,
              np.ndarray[np.double_t,ndim=2] B):
    cdef np.ndarray[np.double_t,ndim=2] C = A.copy()
    cdef unsigned int n, m
    
    openmp.omp_set_num_threads(4)

    n = A.shape[0]
    m = A.shape[1]
    cdef unsigned int k = B.shape[0]//2
    cdef unsigned int i, j, ai, aj

    for i in cython.parallel.prange(k, n-k, nogil=True):
        for j in range(k, m-k):
            C[i,j] = 0
            for ai in range(0, 2*k+1):
                for aj in range(0, 2*k+1):
                    C[i,j] += A[i+ai-k,j+aj-k]*B[ai,aj]
    return C
```

setup.py:

```
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext
import numpy as np

setup(
    name = 'Moj pierwszy pakiet Cython',
    include_dirs = [np.get_include()],
    ext_modules = [
        Extension("modul_cython_test",
                   ["modul_cython_test.pyx"]
                 ),
        Extension("openmp_test",
                   ["openmp_test.pyx"],
                   extra_compile_args=['-fopenmp'],
                   extra_link_args=['-fopenmp']
                 )
    ],
    cmdclass = {
        "build_ext": build_ext
    }
)

```

`__init__.py`:

```
from . import modul_cython_test
from . import openmp_test
```

In [1]:
import pakiet_cython
import skimage.data
import numpy as np
A = skimage.data.lena().mean(axis=2)
B = np.ones((15,15))
B /= B.sum()
#%timeit pakiet_cython.modul_cython_test.convolve2(A,B)
#%timeit pakiet_cython.openmp_test.convolve3(A,B)

In [ ]:
A = np.random.random((1000,1000))
B = np.ones((101,101))
#C = pakiet_cython.modul_cython_test.convolve2(A,B)
C = pakiet_cython.openmp_test.convolve3(A,B)